# Deeper dive on qcodes doNd functionalities with a focus on arrayed sweeping benefits

In this example notebook, we want to show how the qcodes measurement can be simplified with using qcodes doNd functions. Also. we test 2-dimentional sweeping with a naive point by point sweeping over x and y axis and measure z axis values compared to sweeping over x axis and considering y axis as an array of numbers to measure z axis values. The array method can be useful in buffered data acquisitions from instruments that have buffer to write data. Then, the measurement time can be enhanced significantly.

## Necessary imports

In [1]:
from qcodes import initialise_or_create_database_at
import qcodes as qc
import time
import numpy as np
from qcodes import load_or_create_experiment, Measurement, Parameter
from qcodes.utils.dataset import doNd

## Creating a database or loading it

In [2]:
initialise_or_create_database_at('test.db')

Open a cmd or shell and run plottr-inspectr in the activated qcodes environment (assuming you already installed plottr in that 
environment). Then load the created database in the plottr window and check the the box "Auto-plot new". This is an 
encourged data visualization tool for qcodes datasets.

## point by point 2D sweep over x and y

In [3]:
# This cell (#1) is a very naive way of generating an example 2d plot in plottr and time it. This uses single sweep points
# for both x and y directions, so it is quit slow.

xvals = np.linspace(-5, 5, 51)
yvals = np.linspace(-5, 5, 51)
xx, yy = np.meshgrid(xvals, yvals, indexing='ij')
zz = np.cos(xx) * np.sin(yy)

def very_simple_2d_sweep():
    for i, x in enumerate(xvals):
        for j, y in enumerate(yvals):
            yield x, y, zz[i, j]

# configure the qcodes setup
x = Parameter('x')
y = Parameter('y')
z = Parameter('z')
station = qc.Station(x, y, z)
exp = load_or_create_experiment('very_simple_2d_sweep', sample_name='no sample')

# set up the measurement
meas = Measurement(exp, station)
meas.register_parameter(x)
meas.register_parameter(y)
meas.register_parameter(z, setpoints=(x, y))
meas.write_period = 2

# and start recording
t1 = time.time()
with meas.run() as datasaver:
    for xval, yval, zval in very_simple_2d_sweep():
        datasaver.add_result(
            ('x', xval),
            ('y', yval),
            ('z', zval),
        )
        time.sleep(0.1)
t2 = time.time()
print(f'Meaurement took {t2-t1} seconds')

Starting experimental run with id: 93. 
Meaurement took 286.6251063346863 seconds


## Sweeping over x and considering y as an array

In [4]:
# This cell (#2) is for producing the same plot in the above cell with a difference that it uses arrays instead of single
# points for y direction, which means that it is much faster than cell #1.

xvals = np.linspace(-5, 5, 51)
yvals = np.linspace(-5, 5, 51)
xx, yy = np.meshgrid(xvals, yvals, indexing='ij')
zz = np.cos(xx) * np.cos(yy)

def simple_2d_sweep():
    for i, x in enumerate(xvals):
        yield x, yy[i, :], zz[i, :]

# configure qcodes setup
x = Parameter('x')
y = Parameter('y')
z = Parameter('z')
station = qc.Station(x, y, z)
exp = load_or_create_experiment('simple_2d_sweep', sample_name='no sample')

# set up measurement
meas = Measurement(exp, station)
meas.register_parameter(x)
meas.register_parameter(y)
meas.register_parameter(z, setpoints=(x, y))
meas.write_period = 2

# start measuring
t1 = time.time()
with meas.run() as datasaver:
    for xval, yval, zval in simple_2d_sweep():
        datasaver.add_result(
            (x, xval),
            (y, yval),
            (z, zval),
        )
        time.sleep(0.2)

t2 = time.time()
print(f'Meaurement took {t2-t1} seconds')

Starting experimental run with id: 94. 
Meaurement took 10.624388217926025 seconds


## point by point 2D sweep over x and y using qcodes do2d

In [5]:
# This cell (#3) is for producing the same result as the cell #1 with the differnce that it uses qcodes do2d function for
# simplyfing measurement setup. 

def very_simple_2d_sweep_do2d():
    return np.cos(x()) * np.sin(y())

# configure the qcodes setup
x = Parameter('x', set_cmd=None)
y = Parameter('y', set_cmd=None)
z = Parameter('z', get_cmd=lambda:float(very_simple_2d_sweep_do2d()))
station = qc.Station(x, y, z)
exp = load_or_create_experiment('very_simple_2d_sweep_do2d', sample_name='no sample')

t1 = time.time()
doNd.do2d(x, -5, 5, 51, 0.1, y, -5, 5, 51, 0.1, z, do_plot=False)
t2 = time.time()
print(f'Meaurement took {t2-t1} seconds')

Starting experimental run with id: 95. 
Meaurement took 294.2648720741272 seconds


## Sweeping over x and considering y as an array and using qcodes do1d

In [6]:
# This cell (#4) is for producing the same result as the cell #2 with the differnce that it uses qcodes do1d function for
# simplyfing measurement setup. So, it generates the same result as cell #3 with a much faster rate. 

        
def simple_2d_sweep_do1d():
        return np.cos(x()) * np.sin(y())


# configure qcodes setup
x = Parameter('x', set_cmd=None)
y = Parameter('y', get_cmd=lambda: np.linspace(-5, 5, 51))
z = Parameter('z', get_cmd=lambda: simple_2d_sweep_do1d())

station = qc.Station(x, y, z)
exp = load_or_create_experiment('simple_2d_sweep_do1d', sample_name='no sample')

t1 = time.time()
doNd.do1d(x, -5, 5, 51, 0.2, z, additional_setpoints = [y], do_plot=False)
t2 = time.time()
print(f'Meaurement took {t2-t1} seconds')

Starting experimental run with id: 96. 
Meaurement took 10.48732614517212 seconds
